## Setup

Import the necessary evaluation functions.

In [ ]:
from eval import (
    relevance_to_query_score,
    retrieval_groundedness_score,
    evaluate_rag_response,
    print_evaluation_results
)

## Example Question

We'll use the same complex question about dokumentavgift to demonstrate evaluation.

In [ ]:
question = "Hei, jeg lurer på når man slipper å betale dokumentavgift hvis bolig eller eiendom overføres, for eksempel hvis samboere går fra hverandre eller en av dem dør, hvordan det er hvis man arver fast eiendom, og om det også gjelder dersom man får eiendom gjennom testament ved privat skifte?"

print("Spørsmål:", question)

## Example 1: Good Answer (Well-grounded and Relevant)

Let's evaluate a good answer that is both relevant to the question and grounded in the sources.

In [ ]:
# A good answer that addresses all aspects of the question
good_answer = """
Det er flere situasjoner hvor man slipper å betale dokumentavgift:

1. Ved arv av fast eiendom er det fritatt for dokumentavgift
2. Ved skifte av dødsbo, inkludert privat skifte, er det fritatt for dokumentavgift
3. Når samboere går fra hverandre er det normalt ikke fritatt, men ved død av en samboer 
   kan det være fritaksregler som gjelder
4. Testament ved privat skifte følger samme regler som ordinær arv og er fritatt for dokumentavgift
"""

# Relevant sources that support the answer
good_sources = [
    {
        "text": "Dokumentavgift skal ikke betales ved tinglysing av dokumenter som gjelder arv eller skifte av dødsbo, herunder privat skifte.",
        "source": "dokumentavgiftsloven.pdf",
        "retrieval_path": "chunk_15"
    },
    {
        "text": "Fritaksbestemmelsen gjelder både ved arv etter lov og ved testament. Dette omfatter også privat skifte av dødsbo.",
        "source": "dokumentavgiftsloven.pdf",
        "retrieval_path": "chunk_23"
    },
    {
        "text": "Ved overdragelse mellom samboere gjelder de ordinære reglene for dokumentavgift, med mindre overdragelsen skjer som følge av dødsfall.",
        "source": "veiledning_dokumentavgift.pdf",
        "retrieval_path": "chunk_42"
    }
]

print("\n" + "="*80)
print("EKSEMPEL 1: Godt svar (relevant og forankret)")
print("="*80)

results_good = print_evaluation_results(
    question=question,
    answer=good_answer,
    sources=good_sources,
    threshold=4.0
)

## Example 2: Irrelevant Answer

Let's see what happens when the answer doesn't address the question.

In [ ]:
irrelevant_answer = """
I Norge har vi et progressivt skattesystem hvor de som tjener mer betaler mer i skatt.
Dette bidrar til omfordeling og finansiering av velferdsordninger som helsevesen og utdanning.
"""

print("\n" + "="*80)
print("EKSEMPEL 2: Irrelevant svar")
print("="*80)

results_irrelevant = print_evaluation_results(
    question=question,
    answer=irrelevant_answer,
    sources=good_sources,
    threshold=4.0
)

## Example 3: Not Grounded in Sources

What if the answer is relevant but not supported by the provided sources?

In [ ]:
ungrounded_answer = """
Du slipper å betale dokumentavgift ved arv og ved skifte av dødsbo, inkludert testament ved privat skifte.
Samboere som går fra hverandre må normalt betale dokumentavgift ved overdragelse av eiendom.
"""

# Sources that don't support the answer
unrelated_sources = [
    {
        "text": "Eiendomsmegling reguleres av eiendomsmeglingsloven som stiller krav til meglers kompetanse og opptreden.",
        "source": "eiendomsmeglingsloven.pdf",
        "retrieval_path": "chunk_8"
    },
    {
        "text": "Kjøp og salg av bolig er den største økonomiske transaksjonen for de fleste nordmenn.",
        "source": "bolighandel_guide.pdf",
        "retrieval_path": "chunk_12"
    }
]

print("\n" + "="*80)
print("EKSEMPEL 3: Svar ikke forankret i kilder")
print("="*80)

results_ungrounded = print_evaluation_results(
    question=question,
    answer=ungrounded_answer,
    sources=unrelated_sources,
    threshold=4.0
)

## Example 4: Partially Relevant Answer

An answer that only addresses part of the question.

In [ ]:
partial_answer = """
Ved arv av fast eiendom er det fritatt for dokumentavgift.
"""

partial_sources = [
    {
        "text": "Dokumentavgift skal ikke betales ved tinglysing av dokumenter som gjelder arv.",
        "source": "dokumentavgiftsloven.pdf",
        "retrieval_path": "chunk_15"
    }
]

print("\n" + "="*80)
print("EKSEMPEL 4: Delvis relevant svar")
print("="*80)

results_partial = print_evaluation_results(
    question=question,
    answer=partial_answer,
    sources=partial_sources,
    threshold=4.0
)

## Compare Results

Let's create a summary comparison of all evaluations.

In [ ]:
import pandas as pd

# Collect all results
comparison_data = [
    {
        "Eksempel": "1. Godt svar",
        "Relevans": results_good["relevance_score"],
        "Forankring": results_good["groundedness_score"],
        "Bestått": "✅" if results_good["pass_all"] else "❌"
    },
    {
        "Eksempel": "2. Irrelevant",
        "Relevans": results_irrelevant["relevance_score"],
        "Forankring": results_irrelevant["groundedness_score"],
        "Bestått": "✅" if results_irrelevant["pass_all"] else "❌"
    },
    {
        "Eksempel": "3. Ikke forankret",
        "Relevans": results_ungrounded["relevance_score"],
        "Forankring": results_ungrounded["groundedness_score"],
        "Bestått": "✅" if results_ungrounded["pass_all"] else "❌"
    },
    {
        "Eksempel": "4. Delvis relevant",
        "Relevans": results_partial["relevance_score"],
        "Forankring": results_partial["groundedness_score"],
        "Bestått": "✅" if results_partial["pass_all"] else "❌"
    }
]

comparison_df = pd.DataFrame(comparison_data)

print("\n" + "="*80)
print("SAMMENLIGNING AV ALLE EKSEMPLER")
print("="*80)
print("\nScore-skala: 0.0 (dårlig) til 5.0 (utmerket)")
print("Terskel for bestått: 4.0\n")
print(comparison_df.to_string(index=False))

## Individual Metrics

You can also evaluate relevance and groundedness separately.

In [ ]:
# Evaluate only relevance
print("\nRelevance only:")
relevance_score = relevance_to_query_score(
    question=question,
    answer=good_answer
)
print(f"Relevance score: {relevance_score}/5.0")

# Evaluate only groundedness
print("\nGroundedness only:")
groundedness_score = retrieval_groundedness_score(
    answer=good_answer,
    sources=good_sources
)
print(f"Groundedness score: {groundedness_score}/5.0")

## Summary

### Key Takeaways

**Relevance (0-5)**:
- Measures how well the answer addresses the question
- 0 = completely irrelevant
- 3 = partially relevant but incomplete
- 5 = direct and precise answer

**Groundedness (0-5)**:
- Measures whether claims can be verified in sources
- 0 = not grounded at all
- 3 = partially grounded
- 5 = clearly grounded in sources

### Best Practices

1. **Set appropriate thresholds**: Default is 4.0, but adjust based on your use case
2. **Evaluate both metrics**: A good answer must be both relevant AND grounded
3. **Iterate on retrieval**: Low groundedness often indicates poor retrieval
4. **Improve prompts**: Low relevance may indicate issues with the generation prompt

### Using in Production

LLM-as-a-judge evaluation is useful for:
- Testing new retrieval strategies
- Regression testing when updating the system
- Creating evaluation datasets
- Monitoring production quality (sample-based)